In [1]:
import os, time, sys
import numpy as np
np.random.bit_generator = np.random._bit_generator
import torch
from torchvision import transforms, datasets
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
# import imgaug
# from AdaBound import adabound
device = "cuda"

# Se_Net3

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class Sq_Ex_Block(nn.Module):
    def __init__(self, in_ch, r=16):
        super(Sq_Ex_Block, self).__init__()
        self.se = nn.Sequential(
            GlobalAvgPool(),
            nn.Linear(in_ch, in_ch//r),
            nn.ReLU(inplace=True),
            nn.Linear(in_ch//r, in_ch),
            nn.Sigmoid()
        )

    def forward(self, x):
        se_weight = self.se(x).unsqueeze(-1).unsqueeze(-1)
#         print(f'x:{x.sum()}, x_se:{x.mul(se_weight).sum()}')
        return x.mul(se_weight)

class GlobalAvgPool(nn.Module):
    def __init__(self):
        super(GlobalAvgPool, self).__init__()
    def forward(self, x):
        return x.view(*(x.shape[:-2]),-1).mean(-1)

class SE_Net3(nn.Module):
    def __init__(self,in_channels):
        super(SE_Net3,self).__init__()
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, 
        #                dilation=1, groups=1, bias=True, padding_mode='zeros')
        self.c1 = nn.Conv2d(in_channels=in_channels, out_channels=64,kernel_size=3,stride=1,padding=0)
        self.bn1 = nn.BatchNorm2d(num_features=64,eps=1e-3,momentum=0.01)
        self.c2 = nn.Conv2d(64,64,3,1,0)
        self.bn2 = nn.BatchNorm2d(64,1e-3,0.01)
        self.c3 = nn.Conv2d(64,64,3,1,1)
        self.bn3 = nn.BatchNorm2d(64,1e-3,0.01)
        self.c4 = nn.Conv2d(64,64,5,1,2)
        self.bn4 = nn.BatchNorm2d(64,1e-3,0.01)        
        
        self.m1 = nn.MaxPool2d((3,3))
        self.d1 = nn.Dropout(0.5)
        
        self.c5 = nn.Conv2d(64,128,3,1,0)
        self.bn5 = nn.BatchNorm2d(128,1e-3,0.01)
        self.c6 = nn.Conv2d(128,128,3,1,0)
        self.bn6 = nn.BatchNorm2d(128,1e-3,0.01)
        self.c7 = nn.Conv2d(128,128,3,1,1)
        self.bn7 = nn.BatchNorm2d(128,1e-3,0.01)
        self.c8 = nn.Conv2d(128,128,5,1,2)
        self.bn8 = nn.BatchNorm2d(128,1e-3,0.01)
        
        self.m2 = nn.MaxPool2d((2,3))
        self.d2 = nn.Dropout(0.5)
        
        self.c9 = nn.Conv2d(128,256,3,1,0)
        self.bn9 = nn.BatchNorm2d(256,1e-3,0.01)
        self.c10 = nn.Conv2d(256,256,3,1,1)
        self.bn10 = nn.BatchNorm2d(256,1e-3,0.01)
        
        self.se1 = Sq_Ex_Block(in_ch=256,r=16)
        self.m3 = nn.MaxPool2d((2,2))
        self.d3 = nn.Dropout(0.5)

        self.fc1 = nn.Linear(256*3*5,256)
        self.bn11 = nn.BatchNorm1d(256,1e-3,0.01)
        self.out_root = nn.Linear(256,168)
        self.out_vowel = nn.Linear(256,11)
        self.out_constant = nn.Linear(256,7)
        
#         self.init_linear_weights()
        
    def forward(self,x):
        x = self.bn1(F.leaky_relu(self.c1(x),0.05))
        x = self.bn2(F.leaky_relu(self.c2(x),0.05))
        x = self.bn3(F.leaky_relu(self.c3(x),0.05))
        x = self.bn4(F.leaky_relu(self.c4(x),0.05))
        x = self.d1(self.m1(x))
        
        x = self.bn5(F.leaky_relu(self.c5(x),0.05))
        x = self.bn6(F.leaky_relu(self.c6(x),0.05))
        x = self.bn7(F.leaky_relu(self.c7(x),0.05))
        x = self.bn8(F.leaky_relu(self.c8(x),0.05))
        x = self.d2(self.m2(x))
        
        x = self.bn9(F.leaky_relu(self.c9(x),0.05))
        x = self.bn10(F.leaky_relu(self.c10(x),0.05))
        x = self.se1(x)
        x = self.d3(self.m3(x))
        
#         print(x.size())   #Determine fc1 input size
        
        x = x.view(-1, 256*3*5) #reshape
        x = self.bn11(F.leaky_relu(self.fc1(x),0.05))
        
        
        return self.out_root(x),self.out_vowel(x),self.out_constant(x)
    
#     def init_linear_weights(self):
#         nn.init.kaiming_normal_(self.fc1.weight, mode='fan_in')  #default mode: fan_in
#         nn.init.kaiming_normal_(self.out_root.weight, mode='fan_in')
#         nn.init.kaiming_normal_(self.out_vowel.weight, mode='fan_in')
#         nn.init.kaiming_normal_(self.out_constant.weight, mode='fan_in')


# Data preprocessing

In [20]:
def prepare_image(datadir, featherdir, data_type='train',
                  submission=False, indices=[0, 1, 2, 3]):
    assert data_type in ['train', 'test']
    if submission:
        image_df_list = [pd.read_parquet('{}/{}_image_data_{}.parquet'.format(datadir,data_type,i)) for i in indices]
    else:
        image_df_list = [pd.read_feather('{}/{}_image_data_{}.feather'.format(featherdir,data_type,i)) for i in indices]

    print('image_df_list', len(image_df_list))
    
    HEIGHT = 137
    WIDTH = 236
    
    images = [df.iloc[:, 1:].values.reshape(-1, HEIGHT, WIDTH) for df in image_df_list]
    image_ids = [df.iloc[:,0].values for df in image_df_list]
    
#     print(np.shape(image_ids))  #(4,3)
#     print(image_ids)
    
    del image_df_list
#     gc.collect()
    images = np.concatenate(images, axis=0)
    image_ids = np.concatenate(image_ids, axis=0)
    
    return images, image_ids


# test_pd = pd.read_csv('./dataset/test.csv')
# test_labels = test_pd[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].values
datadir = featherdir = "./dataset/"
tmp_indices = [0, 1, 2, 3]
# tmp_indices = [0]
test_images, test_img_id = prepare_image(datadir, featherdir, data_type='test', submission=True, indices=tmp_indices)

image_df_list 4


# Trans and Dataset definition

In [22]:
from imgaug import augmenters as iaa
class ImgAugTransform:
    def __init__(self):
        self.aug = iaa.Sequential([
        iaa.Invert(1),
    ])
    def __call__(self, img, mask=None):
        img = np.array(img)        
        return self.aug.augment_image(image=img)

trans_test = transforms.Compose([
        transforms.Resize((64,128)), #For resnet
        ImgAugTransform(),
        lambda x: Image.fromarray(x),
        transforms.ToTensor(),  #Take Image as input and convert to tensor with value from 0 to1  
])

class TestDataset(Dataset):
    def __init__(self,data_len=None):
        self.data = test_images
        self.image_id = test_img_id
        print("data shape:", np.shape(self.data))
        self.transform = trans_test
        if data_len == None:
            self.len = len(self.data)
        
    def __getitem__(self, idx):
        img = np.uint8(self.data[idx]) #(137,236), value: 0~255
        img = Image.fromarray(img)
        img = self.transform(img)     #value: 0~1, shape:(1,h,w)
        image_id = self.image_id[idx]
        return img, image_id

    def __len__(self):
        return self.len
    

# Get train and val loaders

In [23]:
batch_size = 4
num_workers = 12

test_dataset = TestDataset(data_len=None)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
# mean, std = get_dataset_mean_std(test_loader)
# print("test distribution:",mean, std)


data shape: (12, 137, 236)


# Ensemble inference

In [26]:
components = ['consonant_diacritic', 'grapheme_root', 'vowel_diacritic']
pred_dict = {}
# test_pd = pd.read_csv('./dataset/test.csv')


target=[] # model predictions placeholder
row_id=[] # row_id place holder

ensemble_root = "./Bengali_saved_model/ensemble_adam"
ensemble_models = []

data_num = 0
acc = 0 

for file_name in os.listdir(ensemble_root):
    if file_name.find("Fold") == -1:
        continue
    model = SE_Net3(in_channels=1)
    model.cuda()
    model.load_state_dict(torch.load("{}/{}".format(ensemble_root,file_name)))
    model.eval()
    ensemble_models.append(model)
    

model_num = len(ensemble_models)
print("len of models:",len(ensemble_models))    
result = np.array([])
label = np.array([])    
with torch.no_grad():
    for idx, data in enumerate(test_loader):
        img, img_id = data
        img = img.to(device)
        
#         print(np.shape(img_id))
#         print(img_id)
        
        ###Average Ensemble
        pred_list_root = torch.Tensor([]).to(device)
        pred_list_vow = torch.Tensor([]).to(device)
        pred_list_const = torch.Tensor([]).to(device)
        
        for i in range(model_num):
            pred_root, pred_vow, pred_const = ensemble_models[i](img) #(batch_num, label_num)
            pred_list_root = torch.cat((pred_list_root,pred_root.unsqueeze(2)),dim=2)      #pred_list: (batch_num,168,model_num)
            pred_list_vow = torch.cat((pred_list_vow,pred_vow.unsqueeze(2)),dim=2)         #pred_list: (batch_num,11,model_num)
            pred_list_const = torch.cat((pred_list_const,pred_const.unsqueeze(2)),dim=2)   #pred_list: (batch_num,7,model_num)
        
        pred_root = torch.mean(pred_list_root,dim=2)   #(batch,10)
        pred_vow = torch.mean(pred_list_vow,dim=2)   #(batch,10)
        pred_const = torch.mean(pred_list_const,dim=2)   #(batch,10)
        
        _,pred_class_root = torch.max(pred_root.data, 1)   #(batch_num,)        
        _,pred_class_vow = torch.max(pred_vow.data, 1)   #(batch_num,)        
        _,pred_class_const = torch.max(pred_const.data, 1)   #(batch_num,)        
        
#         pred_dict['consonant_diacritic'] = pred_class_const
#         pred_dict['grapheme_root'] = pred_class_root
#         pred_dict['vowel_diacritic'] = pred_class_vow
        
        for i,test_id in enumerate(img_id):
            row_id.append(test_id+'_consonant_diacritic')
            row_id.append(test_id+'_grapheme_root')
            row_id.append(test_id+'_vowel_diacritic')
            target.append(pred_class_const[i].cpu().numpy())
            target.append(pred_class_root[i].cpu().numpy())
            target.append(pred_class_vow[i].cpu().numpy())
        
#         print("check")
#         print(np.shape(pred_class_root))
#         print(np.shape(pred_class_vow))
#         print(np.shape(pred_class_const))        
        
        ###Voting Ensemble
#         pred_list = torch.LongTensor([]).to(device)
#         for i in range(model_num):
#             pred = ensemble_models[i](img) #(batch_num,10)
#             _,pred_class = torch.max(pred.data, 1)   #(batch_num,)
#             pred_list = torch.cat((pred_list,pred_class.unsqueeze(1)),dim=1)
#         for i in range(model2_num):
#             pred = ensemble_models2[i](img) #(batch_num,10)
#             _,pred_class = torch.max(pred.data, 1)   #(batch_num,)
#             pred_list = torch.cat((pred_list,pred_class.unsqueeze(1)),dim=1)
            
#         pred_class_list = torch.LongTensor([]).to(device)
#         for i in range(img.size(0)):
#             pred_np = pred_list[i].cpu().numpy()
#             unique_class,count = np.unique(pred_np,return_counts=True)
#             unique_class = np.array(unique_class[np.argmax(count)]).reshape(-1)   #unique class shape(1,)
#             class_voted= torch.from_numpy(unique_class).to(device)    #(1,)
#             pred_class_list = torch.cat((pred_class_list,class_voted))    
    
    df_sample = pd.DataFrame(
        {
            'row_id': row_id,
            'target':target
        },
        columns = ['row_id','target'] 
    )
    df_sample.to_csv('submission.csv',index=False)
    df_sample.head()
    

len of models: 1


In [ ]:
print(np.where(label!=result)[0])
print(np.where(label!=result1)[0])

In [ ]:
self.data = global_dataimport torch
import numpy as np

t1 = torch.Tensor([[1,2,3,4],[4,3,2,1],[1,5,3,3]])  #(3,4)
t1 = t1.unsqueeze(2)

t_list = torch.Tensor([])

for i in range(3):
    t_list = torch.cat((t_list,t1),dim=2)

print(t_list.size())
print(t_list)
t_list = torch.mean(t_list,dim=2)
print(t_list.size())
print(t_list)


# n1 = t1.cpu().numpy()

# n1, count = np.unique(n1,return_counts=True,axis=0)
# print(count)
# n1 = np.argmax(count)

